In [1]:
#importing req. Lib.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
#load our data set
data = pd.read_csv('Tweets.csv')
data.head()

#tweet_created column got the date recorts and showing type is object we have to change it of date time format
data['tweet_created'] = pd.to_datetime(data['tweet_created']).dt.date
data['tweet_created'] = pd.to_datetime(data['tweet_created'])

print("Percentage null or na values in df")
((data.isnull() | data.isna()).sum() * 100 / data.index.size).round(2)

del data['tweet_coord']
del data['airline_sentiment_gold']
del data['negativereason_gold']


data.drop(data.loc[data['airline_sentiment']=='neutral'].index, inplace=True)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(data['airline_sentiment'])

data['airline_sentiment_encoded'] = le.transform(data['airline_sentiment'])

# cleaning the tweet text data to apply classification algorithms on it

import re

def tweet_to_words(tweet):
    letters_only = re.sub("[^a-zA-Z]", " ", tweet) 
    words = letters_only.lower().split()
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops]
    return " ".join(meaningful_words)


from nltk.corpus import stopwords

nltk.download('stopwords')
data['clean_tweet'] = data['text'].apply(lambda x: tweet_to_words(x))


### Vectorization
x = data.clean_tweet
y = data.airline_sentiment


x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.25)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(x_train)

# Use the trained to create a document-term matrix from train and test sets
x_train_dtm = vect.transform(x_train)
x_test_dtm = vect.transform(x_test)

vect_tunned = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.1, max_df=0.7, max_features=100)
vect_tunned


from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, FloatType

# Start Spark session
spark = SparkSession.builder.appName('SentimentAnalysis').getOrCreate()

# Convert pandas dataframe to Spark dataframe
sdf = spark.createDataFrame(data)

# Define stages for the pipeline
tokenizer = Tokenizer(inputCol="clean_tweet", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")
labelIndexer = StringIndexer(inputCol="airline_sentiment", outputCol="label")
lr = LogisticRegression(maxIter=10, regParam=0.01)

# Create the pipeline
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, labelIndexer, lr])

# Split the data into train and test sets
(trainingData, testData) = sdf.randomSplit([0.8, 0.2], seed=1234)

# Train the model
model = pipeline.fit(trainingData)

# Make predictions on test data
predictions = model.transform(testData)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

def predict_sentiment(text):
    # Create a local DataFrame with the text
    local_df = spark.createDataFrame([(text,)], ["clean_tweet"])

    # Transform the local DataFrame using the model to make a prediction
    prediction = model.transform(local_df)

    # Extract the predicted label index and probability vector
    predicted_label_index = prediction.select("prediction").collect()[0]["prediction"]
    predicted_probability = prediction.select("probability").collect()[0]["probability"].toArray()

    # Extract the labels from the StringIndexerModel (which is the last but one stage in the pipeline)
    labels = model.stages[-2].labels

    # Map the predicted label index (float) to the corresponding string label
    predicted_label = labels[int(predicted_label_index)]

    # Calculate the confidence
    confidence = float(predicted_probability[int(predicted_label_index)])

    return predicted_label, confidence

# Test the prediction function
text_to_predict = "I had an okay flight with comfortable seats and average customer service"
predicted_sentiment, confidence = predict_sentiment(text_to_predict)
print(f"The predicted sentiment for the text '{text_to_predict}' is: {predicted_sentiment} with confidence {confidence:.2f}")






In [ ]:
Task 1. Prepare 15 slides


1-3 - Introdution of you and me , Introdution of our problem statement, Why the need to use cloud ?
4-5   Dataset Explanation
++ 6-11 - Flow of the project (flowchart), modeling part (accuracy between model),front end part (Website page) and explaining fucntionalities
12-15 - why the need of cloud based platforms for this, deployment, usabilty , future work, references